Imports

In [2]:
# stdlib
import os
from time import perf_counter

# pip
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras import backend as K

# pretrained imagenet ResNets
from classification_models.tfkeras import Classifiers

# local
from utils import get_dirs, train_test_split

Load and pre-process data to be compatible with the ResNet model

In [4]:
DIRS = get_dirs(os.path.abspath('') + os.sep + 'Task2_help.ipynb')
print('\033[1m' + 'Directories:' + '\033[0m')
for dir_name, path in DIRS.items():
    print(f'{dir_name:<7} {path}')

images = np.load(DIRS['data'] + 'images.npy')
labels = np.load(DIRS['data'] + 'labels.npy')

X = np.stack([images]*3, axis=-1)  # (18000 x 150 x 150) -> (18000 x 150 x 150 x 3)
y = labels.astype('int32')

np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(X=X, y=y, test_size=0.2)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# create regression labels
# (4, 30) -> 4.5
# (11, 15) -> 11.25
reg_y_train = (y_train[:, 0] + y_train[:, 1] / 60).astype('float32')
reg_y_test  = (y_test[:, 0] + y_test[:, 1] / 60).astype('float32')

# create classification labels where the 12 hours are split into 48 quarter-hour bins
# (4, 30) -> 18
# (11, 15) -> 45
class_y_train = (y_train[:, 0] * 4 + y_train[:, 1] // 15).astype('float32')
class_y_test  = (y_test[:, 0] * 4 + y_test[:, 1] // 15).astype('float32')
# one hot encoding
class_y_train = keras.utils.to_categorical(class_y_train, num_classes=48)
class_y_test = keras.utils.to_categorical(class_y_test, num_classes=48)

#create classifcation labels where the 12 hours are split 12 hours and the minutes are split into 12 bins

classhours_y_train = (y_train[:, 0]).astype('float32')
classhours_y_test  = (y_test[:, 0]).astype('float32')
classminutes_y_train = (y_train[:, 1] // 5).astype('float32')
classminutes_y_test  = (y_test[:, 1] // 5).astype('float32')
# one hot encoding
classhours_y_train = keras.utils.to_categorical(classhours_y_train, num_classes=12)
classhours_y_test = keras.utils.to_categorical(classhours_y_test, num_classes=12)
classminutes_y_train = keras.utils.to_categorical(classminutes_y_train, num_classes=12)
classminutes_y_test = keras.utils.to_categorical(classminutes_y_test, num_classes=12)


del images, labels, X, y

print('\n' + '\033[1m' + 'Data:' + '\033[0m')
for name, arr in zip(['X_train', 'X_test', 'y_train', 'y_test'], [X_train, X_test, y_train, y_test]):
    print(f'{name:<7} {str(arr.shape):<20} {arr.dtype}')

Directories:
data    c:\Users\wille\Documents\GitHub\MCG\data\
results c:\Users\wille\Documents\GitHub\MCG\results\
csv     c:\Users\wille\Documents\GitHub\MCG\results\csv\
plots   c:\Users\wille\Documents\GitHub\MCG\results\plots\
figs    c:\Users\wille\Documents\GitHub\MCG\results\figs\
models  c:\Users\wille\Documents\GitHub\MCG\results\models\

Data:
X_train (14400, 150, 150, 3) float32
X_test  (3600, 150, 150, 3)  float32
y_train (14400, 2)           int32
y_test  (3600, 2)            int32


In [5]:
def cs_loss(y_true, y_pred):
    return K.minimum(K.abs(y_true - y_pred), K.abs(K.ones_like(y_pred) * 12 - K.abs(y_true - y_pred)))

def sigmoid_12(x):
    # sigmoid function with range 0 to 12
    return 12 / (1 + K.exp(-x))


def build_resnet() -> keras.Model:
    """
    ResNet model pretrained on 1000 class imagenet data set
    ResNet 34 is loaded (34 deep convolutional layers)
    The top of the ResNet model is excluded
    Three layers are added to end of ResNet, a GAPooling and two dense layers
    Last layer has linear activation to predict a continuous regression value
    """

    # load base model
    ResNet, _ = Classifiers.get('resnet34')
    base_model = ResNet(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
    
    # add top layers
    pooled = layers.GlobalAveragePooling2D()(base_model.output)
    flatten = layers.Dense(256, activation='relu')(pooled)
    # final layer gets 48 nodes with softmax
    final1 = layers.Dense(12, activation='softmax')(flatten)
    final2 = layers.Dense(12, activation='softmax')(flatten)


    # create and compile Keras model object
    model = keras.Model(inputs=base_model.input, outputs=[final1, final2])
    model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy', 'accuracy'])
    
    return model

In [6]:
model = build_resnet()

df = pd.DataFrame(columns=['mae', 'totall_loss', 'hourloss', 'minuteloss' 'train_time', 'test_time'])

folds = 5
fold_size_train = X_train.shape[0] // folds
fold_size_test = X_test.shape[0] // folds

for fold in range(folds):
    print(f'\n---\n' + '\033[1m' + f'Fold {fold+1}/{folds}' + '\033[0m' + '\n')
    # slice the arrays
    X_train_fold = X_train[fold * fold_size_train : (fold + 1) * fold_size_train]
    X_test_fold = X_test[fold * fold_size_test : (fold + 1) * fold_size_test]
    y_train_fold_hours = classhours_y_train[fold * fold_size_train : (fold + 1) * fold_size_train]
    y_train_fold_minutes = classminutes_y_train[fold * fold_size_train : (fold + 1) * fold_size_train]
    y_test_fold_hours = classhours_y_test[fold * fold_size_test : (fold + 1) * fold_size_test]
    y_test_fold_minutes = classminutes_y_test[fold * fold_size_test : (fold + 1) * fold_size_test]

    # train the model
    tic_train = perf_counter()
    history = model.fit(X_train_fold, [y_train_fold_hours, y_train_fold_minutes], epochs=10, batch_size=32, verbose=1)
    toc_train = perf_counter()

    # evaluate the model
    tic_test = perf_counter()
    totalLoss, hourloss, minuteloss = model.evaluate(X_test_fold, [y_test_fold_hours, y_test_fold_minutes], verbose=0)
    toc_test = perf_counter()

    # store the results
    df.loc[fold] = [mae, totalLoss, hourloss, minuteloss, toc_train - tic_train, toc_test - tic_test]


15548416/85521592 [====>.........................] - ETA: 10s

KeyboardInterrupt: 

In [ ]:
# save results
model.save(DIRS['models'] + 'clock_class_48.h5')
df.to_csv(DIRS['csv'] + 'clock_class_48.csv', index=False)
df

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(class_y_test, axis=1)

cm = confusion_matrix(y_test, y_pred)
accuracy = np.sum(y_pred == y_test) / y_test.shape[0]
print(f'Accuracy: {accuracy:.2%}')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cm, cmap='Reds')
# colorbar
cbar = fig.colorbar(ax.images[0], shrink=0.5)
cbar.ax.set_ylabel('Count')
# set ticks to represent hours in range 0 - 11:45
ticks = np.arange(0, 48, 4)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels([f'{i:02d}:00' for i in range(0, 12)])
ax.set_yticklabels([f'{i:02d}:00' for i in range(0, 12)])
# set labels
ax.set_title('Confusion Matrix 48 Class ResNet')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
# rotate x labels
plt.setp(ax.get_xticklabels(), rotation=-45, ha='left', rotation_mode='anchor')
fig.tight_layout()
fig.savefig(DIRS['plots'] + 'cm_48_class_resnet.png', dpi=300)

In [ ]:
# get predictions on some random samples and plot the image along with true and predicted time
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
for ax in axes:
    i = np.random.randint(0, X_test.shape[0])
    ax.imshow(X_test[i])
    min_to_human_time = lambda m: f'{m // 60:02d}:{m % 60:02d}'
    pred = y_pred[i] * 15
    true = y_test[i] * 15
    ax.set_title(f'Pred: {min_to_human_time(pred)} - True: {min_to_human_time(true)}')
    ax.axis('off')
